In [1]:
import os

In [2]:
!pip install opendatasets

In [106]:
import opendatasets as od
import pandas as pd
from sklearn.model_selection import train_test_split

In [107]:
od.download('https://www.kaggle.com/datasets/gauravduttakiit/resume-dataset')

Skipping, found downloaded files in "./resume-dataset" (use force=True to force download)


In [108]:
data=pd.read_csv('/content/resume-dataset/UpdatedResumeDataSet.csv')

In [109]:
data.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [110]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Label'] = le.fit_transform(data['Category'])

In [111]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

data['clean_text'] = data['Resume'].apply(clean_text)


In [112]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_NUM_WORDS = 10000
MAX_SEQ_LEN = 500

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(data['clean_text'])

sequences = tokenizer.texts_to_sequences(data['clean_text'])
padded = pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')


In [113]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded, data['Label'], test_size=0.2, stratify=data['Label'])


In [117]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.layers import Bidirectional

model = Sequential([
    Embedding(MAX_NUM_WORDS, 128),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(le.classes_), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [120]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 628ms/step - accuracy: 0.8774 - loss: 0.5207 - val_accuracy: 0.9481 - val_loss: 0.3517
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 624ms/step - accuracy: 0.8794 - loss: 0.4434 - val_accuracy: 0.9481 - val_loss: 0.2811
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 624ms/step - accuracy: 0.9270 - loss: 0.3161 - val_accuracy: 0.9675 - val_loss: 0.1792
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 625ms/step - accuracy: 0.9336 - loss: 0.2702 - val_accuracy: 0.9805 - val_loss: 0.1379
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 625ms/step - accuracy: 0.9657 - loss: 0.1908 - val_accuracy: 0.9935 - val_loss: 0.0997
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 625ms/step - accuracy: 0.9614 - loss: 0.1840 - val_accuracy: 0.9675 - val_loss: 0.1260
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 623ms/step - accuracy: 0.9759 - loss: 0.1528 - val_accuracy: 0.9870 - val_loss: 0.0745
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 21s 626ms/step - accuracy: 0.9902 - loss: 0.0971 - val_accu

In [121]:
model.evaluate(X_test, y_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step - accuracy: 1.0000 - loss: 0.0029


[0.0028576331678777933, 1.0]

In [142]:
text='''Career Objective
Dedicated and detail-oriented Java Developer with 3+ years of experience in designing, developing, and maintaining scalable Java applications. Seeking to contribute my expertise to a growth-oriented company where I can apply my skills and continue to grow.

Skills
Core Java, Java 8+

Spring, Spring Boot, Hibernate

RESTful APIs, Microservices

Maven, Jenkins, Git

MySQL, Oracle, MongoDB

HTML, CSS, JavaScript

JUnit, Mockito

Agile, Scrum methodologies

Education
Bachelor of Technology in Computer Science
ABC Institute of Technology, Hyderabad
2016 – 2020 | CGPA: 8.5/10

Professional Experience
Java Developer
XYZ Solutions Pvt. Ltd., Bangalore
Aug 2020 – Present

Designed and implemented REST APIs using Spring Boot.

Developed microservices to handle business logic for order management system.

Worked with MySQL and MongoDB for database operations.

Participated in daily stand-ups and sprint planning in Agile environment.

Integrated third-party APIs and worked with JSON/XML data formats.

Software Developer Intern
TechLabs, Hyderabad
Jan 2020 – Jun 2020

Assisted in building a customer management web application using Java and Spring MVC.

Wrote unit test cases using JUnit.

Debugged and fixed bugs reported during testing.

Projects
E-commerce Backend API (GitHub)
Built a full-stack backend API using Spring Boot and MySQL, featuring user authentication, product catalog, and order processing.

Chat Application (GitHub)
Developed a real-time chat application using Java WebSockets and deployed it using Tomcat server.

Certifications
Oracle Certified Associate, Java SE 8 Programmer

Spring Framework – Udemy Certification

Languages
English (Fluent)

Hindi (Native)

'''

In [143]:
c=clean_text(text)

In [144]:
c

'career objective dedicated and detailoriented java developer with 3 years of experience in designing developing and maintaining scalable java applications seeking to contribute my expertise to a growthoriented company where i can apply my skills and continue to grow skills core java java 8 spring spring boot hibernate restful apis microservices maven jenkins git mysql oracle mongodb html css javascript junit mockito agile scrum methodologies education bachelor of technology in computer science abc institute of technology hyderabad 2016  2020  cgpa 8510 professional experience java developer xyz solutions pvt ltd bangalore aug 2020  present designed and implemented rest apis using spring boot developed microservices to handle business logic for order management system worked with mysql and mongodb for database operations participated in daily standups and sprint planning in agile environment integrated thirdparty apis and worked with jsonxml data formats software developer intern techl

In [145]:
se=tokenizer.texts_to_sequences([c])
padd=pad_sequences(se, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')

In [146]:
pr=model.predict(padd)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


In [147]:
pr

array([[1.8844053e-06, 1.9084663e-03, 2.5122135e-03, 1.2032301e-01,
        5.0752966e-05, 8.1325757e-05, 2.3140912e-05, 1.9488620e-02,
        3.2548243e-03, 5.9449684e-04, 2.9503130e-05, 5.0619046e-06,
        4.4207660e-05, 4.8126769e-04, 2.6027532e-05, 7.7536273e-01,
        3.3169359e-05, 7.8673661e-04, 3.8210569e-06, 6.6107852e-05,
        5.5557764e-05, 1.9685302e-07, 4.0286764e-06, 7.2532319e-02,
        2.3305912e-03]], dtype=float32)

In [148]:
p=np.argmax(pr)

In [149]:
p

np.int64(15)

In [150]:
def decode_label(index):
    return le.inverse_transform([index])[0]

In [151]:
decode_label(p)

'Java Developer'

In [152]:
import pickle

In [153]:
with open("labels.pkl", "wb") as f:
    pickle.dump(le, f)

In [154]:
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [156]:
model.save("model.h5")